# 電話番号フィルタリング問題の解説

この問題を段階的に解説し、bash one-linerで解決します。

## 問題の理解

**有効な電話番号の形式:**
1. `(xxx) xxx-xxxx` - カッコ付き形式
2. `xxx-xxx-xxxx` - ハイフン形式

ここで `x` は数字(0-9)を表します。

## 解決策

```bash
#!/bin/bash

# Solution 1: Using grep with extended regex
# Analyze Complexity
# Runtime 63 ms
# Beats 58.79%
# Memory 3.47 MB
# Beats 84.10%

grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt
```

```bash
# Solution 2: Using sed (alternative)
# Analyze Complexity
# Runtime 65 ms
# Beats 45.95%
# Memory 3.42 MB
# Beats 84.10%

sed -n -E '/^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$/p' file.txt
```

```bash
# Solution 3: Using awk (alternative)
# Analyze Complexity
# Runtime 71 ms
# Beats 15.78%
# Memory 3.84 MB
# Beats 3.13%

awk '/^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$/' file.txt
```

## 最もシンプルな解答

```bash
grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt
```

---

## 詳細な図解による解説

### 1. 正規表現パターンの構造

```mermaid
graph TD
    A["正規表現全体<br/>^...OR...$"] --> B["行頭アンカー ^"]
    A --> C["OR演算子 |"]
    A --> D["行末アンカー $"]
    
    C --> E["パターン1<br/>xxx-xxx-xxxx"]
    C --> F["パターン2<br/>(xxx) xxx-xxxx"]
    
    E --> E1["[0-9]{3}"]
    E --> E2["-"]
    E --> E3["[0-9]{3}"]
    E --> E4["-"]
    E --> E5["[0-9]{4}"]
    
    F --> F1["\("]
    F --> F2["[0-9]{3}"]
    F --> F3["\)"]
    F --> F4["スペース"]
    F --> F5["[0-9]{3}"]
    F --> F6["-"]
    F --> F7["[0-9]{4}"]
    
    style A fill:#e1f5ff
    style E fill:#c8e6c9
    style F fill:#fff9c4
```

#### **パターン1の詳細構造**

```mermaid
graph LR
    A["[0-9]{3}"] -->|"例: 987"| B["-"]
    B --> C["[0-9]{3}"]
    C -->|"例: 123"| D["-"]
    D --> E["[0-9]{4}"]
    E -->|"例: 4567"| F["結果: 987-123-4567"]
    
    style A fill:#ffcdd2
    style C fill:#f8bbd0
    style E fill:#e1bee7
    style F fill:#c5cae9
```

#### **パターン2の詳細構造**

```mermaid
graph LR
    A["\("] --> B["[0-9]{3}"]
    B -->|"例: 123"| C["\)"]
    C --> D["スペース"]
    D --> E["[0-9]{3}"]
    E -->|"例: 456"| F["-"]
    F --> G["[0-9]{4}"]
    G -->|"例: 7890"| H["結果: (123) 456-7890"]
    
    style A fill:#ffcdd2
    style B fill:#f8bbd0
    style C fill:#ffcdd2
    style E fill:#e1bee7
    style G fill:#d1c4e9
    style H fill:#c5cae9
```

---

### 2. grep コマンドの動作フロー

```mermaid
flowchart TD
    A["file.txt<br/>行1: 987-123-4567<br/>行2: 123 456 7890<br/>行3: (123) 456-7890"] --> B["grep -E でパターンマッチング開始"]
    
    B --> C1["行1をチェック:<br/>987-123-4567"]
    B --> C2["行2をチェック:<br/>123 456 7890"]
    B --> C3["行3をチェック:<br/>(123) 456-7890"]
    
    C1 --> D1{"パターン1<br/>にマッチ?"}
    D1 -->|"✓ YES"| E1["出力に追加"]
    
    C2 --> D2{"どちらかの<br/>パターンに<br/>マッチ?"}
    D2 -->|"✗ NO<br/>(スペース区切り)"| E2["スキップ"]
    
    C3 --> D3{"パターン2<br/>にマッチ?"}
    D3 -->|"✓ YES"| E3["出力に追加"]
    
    E1 --> F["最終出力"]
    E2 --> F
    E3 --> F
    
    F --> G["987-123-4567<br/>(123) 456-7890"]
    
    style A fill:#e3f2fd
    style D1 fill:#c8e6c9
    style D2 fill:#ffcdd2
    style D3 fill:#c8e6c9
    style E1 fill:#a5d6a7
    style E2 fill:#ef9a9a
    style E3 fill:#a5d6a7
    style G fill:#81c784
```

---

### 3. オプションと構文要素の説明

```mermaid
graph TD
    A["grep -E '^pattern$' file.txt"] --> B["-E オプション"]
    A --> C["^ アンカー"]
    A --> D["$ アンカー"]
    A --> E["file.txt"]
    
    B --> B1["拡張正規表現を有効化<br/>+, ?, |, () が使用可能"]
    C --> C1["行頭にマッチ<br/>余分な前置文字を排除"]
    D --> D1["行末にマッチ<br/>余分な後置文字を排除"]
    E --> E1["入力ファイル<br/>各行を順次処理"]
    
    style A fill:#e1f5ff
    style B fill:#fff9c4
    style C fill:#c8e6c9
    style D fill:#c8e6c9
    style E fill:#ffccbc
```

---

### 4. テストケースの検証フロー

```mermaid
flowchart TD
    A["入力電話番号"] --> B{"形式チェック"}
    
    B -->|"パターン1"| C1["xxx-xxx-xxxx"]
    B -->|"パターン2"| C2["(xxx) xxx-xxxx"]
    B -->|"その他"| C3["無効な形式"]
    
    C1 --> D1{"各部分が<br/>正しい桁数?"}
    D1 -->|"✓ YES"| E1["✓ 有効<br/>例: 987-123-4567"]
    D1 -->|"✗ NO"| F1["✗ 無効<br/>例: 12-345-6789"]
    
    C2 --> D2{"カッコとスペースが<br/>正しい位置?"}
    D2 -->|"✓ YES"| E2["✓ 有効<br/>例: (123) 456-7890"]
    D2 -->|"✗ NO"| F2["✗ 無効<br/>例: (123)456-7890"]
    
    C3 --> F3["✗ 無効<br/>例: 123 456 7890<br/>例: 1234567890"]
    
    style E1 fill:#a5d6a7
    style E2 fill:#a5d6a7
    style F1 fill:#ef9a9a
    style F2 fill:#ef9a9a
    style F3 fill:#ef9a9a
```

---

### 5. 実行例のシーケンス

```mermaid
sequenceDiagram
    participant User
    participant Shell
    participant grep
    participant file.txt
    
    User->>Shell: cat > file.txt
    Shell->>file.txt: 987-123-4567<br/>123 456 7890<br/>(123) 456-7890
    
    User->>Shell: grep -E '^pattern$' file.txt
    Shell->>grep: コマンド実行
    
    grep->>file.txt: 行1を読み込み
    file.txt-->>grep: 987-123-4567
    grep->>grep: パターン1にマッチ ✓
    grep->>Shell: 987-123-4567 を出力
    
    grep->>file.txt: 行2を読み込み
    file.txt-->>grep: 123 456 7890
    grep->>grep: マッチせず ✗
    
    grep->>file.txt: 行3を読み込み
    file.txt-->>grep: (123) 456-7890
    grep->>grep: パターン2にマッチ ✓
    grep->>Shell: (123) 456-7890 を出力
    
    Shell->>User: 987-123-4567<br/>(123) 456-7890
```

---

## まとめ

**最適解:**
```bash
grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt
```

この一行で:
- 2つの有効な形式を検出
- 行頭と行末の厳密なマッチング
- シンプルで効率的な実装

が実現できます!